In [1]:
# Importing required libraries

!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge geopy --yes

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import random
from geopy.geocoders import Nominatim
import folium
from pandas.io.json import json_normalize
from IPython.display import Image 
from IPython.core.display import HTML 

print("Success")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Success


Part 1 -> Creating the required table

In [2]:
# Using BeautifulSoup to scrape data from Wikipedia

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 
response= requests.get(url) 
soup = BeautifulSoup(response.text ,'html.parser')
print("Success")
print(soup)

Success

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"effb7f1c-76f8-4e4a-96e6-cb5115a410a2","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ont

In [3]:
# Creating the Table

tbl = soup.find('table')

listPostalCode   = []
listBorough      = [] 
listNeighbourhood = []
 
td1 = tbl.find_all('td')
for i in range (0, len(td1), 3 ):
    listPostalCode.append(td1[i].text.strip())
    listBorough.append(td1[i+1].text.strip())
    listNeighbourhood.append(td1[i+2].text.strip())
    
    
TorontoDF = pd.DataFrame(data=[listPostalCode, listBorough, listNeighbourhood]).transpose()
TorontoDF.columns = ['PostalCode', 'Borough', 'Neigbourhood']
TorontoDF

,PostalCode,Borough,Neigbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [4]:
TorontoDF['Borough'].replace('Not assigned', np.nan, inplace= True)
TorontoDF.dropna(subset =['Borough'], inplace= True)
TorontoDF.head()

,PostalCode,Borough,Neigbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
TorontoDF1 = TorontoDF.groupby(['PostalCode','Borough'])['Neigbourhood'].apply(', '.join).reset_index()
TorontoDF1.columns = ['PostalCode', 'Borough', 'Neigbourhood']
TorontoDF1

,PostalCode,Borough,Neigbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [6]:
TorontoDF1['Neigbourhood'].replace('Not assigned', TorontoDF1["Borough"], inplace=True) 
TorontoDF1.shape

(103, 3)

Part 2 -> Getting Latitude and Longitude data

In [7]:
TorontoGEO = pd.read_csv('http://cocl.us/Geospatial_data')
TorontoGEO.columns = ['PostalCode', 'Latitude', 'Longitude']
TorontoGEO

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [8]:
TorontoDF2 = pd.merge(TorontoDF1, TorontoGEO, on=['PostalCode'], how='inner')
TorontoDF3 = TorontoDF2[['Borough', 'Neigbourhood', 'PostalCode', 'Latitude', 'Longitude']].copy()
TorontoDF3

,Borough,Neigbourhood,PostalCode,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
...,...,...,...,...,...
98,York,Weston,M9N,43.706876,-79.518188
99,Etobicoke,Westmount,M9P,43.696319,-79.532242
100,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",M9R,43.688905,-79.554724
101,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",M9V,43.739416,-79.588437


In [1]:
# Foursquare API access requirements

clientID = ''
clientSecret = '' 
ver = '20180604'
limit = 30
print('Your credentails:')
print('CLIENT_ID: ' + clientID)
print('CLIENT_SECRET:' + clientSecret)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [10]:
addr = 'Toronto, Canada'
geolocator = Nominatim(user_agent="foursquare_agent")
loc = geolocator.geocode(addr)
longitude = loc.longitude
latitude = loc.latitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


In [11]:
TorontoDF4 = TorontoDF3[TorontoDF1['Borough'].str.contains('Toronto')]

TorontoDF5 = TorontoDF4.reset_index(drop=True)
TorontoDF5

,Borough,Neigbourhood,PostalCode,Latitude,Longitude
0,East Toronto,The Beaches,M4E,43.676357,-79.293031
1,East Toronto,"The Danforth West, Riverdale",M4K,43.679557,-79.352188
2,East Toronto,"India Bazaar, The Beaches West",M4L,43.668999,-79.315572
3,East Toronto,Studio District,M4M,43.659526,-79.340923
4,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790
5,Central Toronto,Davisville North,M4P,43.712751,-79.390197
6,Central Toronto,"North Toronto West, Lawrence Park",M4R,43.715383,-79.405678
7,Central Toronto,Davisville,M4S,43.704324,-79.388790
8,Central Toronto,"Moore Park, Summerhill East",M4T,43.689574,-79.383160
9,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",M4V,43.686412,-79.400049


In [12]:
# Mapping the points

TorontoMap = folium.Map(location=[latitude, longitude], zoom_start=12)
for lati, long, borough, neigborhood in zip(TorontoDF5['Latitude'], TorontoDF5['Longitude'], TorontoDF5['Borough'], TorontoDF5['Neigbourhood']):
    mark = '{}, {}'.format(neigborhood, borough)
    mark = folium.Popup(mark, parse_html=True)
    folium.CircleMarker(
        [lati, long],
        radius=5,
        popup=mark,
        color='blue',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(TorontoMap) 
    
TorontoMap

Part 3 -> Exploring and Clustering

In [13]:
rad = 1000 # Radius covered
lim = 50 # Limiting the number of entries

In [14]:
# Selecting a random neighbourhood

TorontoDF5.loc[2,'Neigbourhood']

'India Bazaar, The Beaches West'

In [15]:
# Getting Latitude and Longitude information

nghLongitude = TorontoDF5.loc[2, 'Longitude']
nghLatitude = TorontoDF5.loc[2, 'Latitude']

print('Longitude -> {} and Latitude -> {}'.format(nghLongitude, nghLatitude))

Longitude -> -79.31557159999998 and Latitude -> 43.6689985


In [16]:
# Accessing Foursquare API

url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    clientID, 
    clientSecret, 
    ver, 
    nghLatitude, 
    nghLongitude, 
    rad, 
    lim)

In [17]:
# Storing data

queryResults = requests.get(url1).json()
queryResults

{'meta': {'code': 200, 'requestId': '5f0c5fe74cfdf626a87b7bf8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'East End',
  'headerFullLocation': 'East End, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 78,
  'suggestedBounds': {'ne': {'lat': 43.67799850900001,
    'lng': -79.30315255189772},
   'sw': {'lat': 43.65999849099999, 'lng': -79.32799064810224}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ade390ff964a5200e7421e3',
       'name': 'System Fitness',
       'location': {'address': '1661 Queen St East',
        'crossStreet': 'at Kingston Rd',
        'lat': 43.667171452103204,
        'lng': -79.31273345707406,
        'labeledLatLngs': [{

In [18]:
# Cleaning and organizing JSON data

def getData(row):
    try:
        data = row['categories']
    except:
        data = row['venue.categories']
    if len(data) == 0:
        return None
    else:
        return data[0]['name']

venues = queryResults['response']['groups'][0]['items']
venuesNorm = json_normalize(venues)
updtCol = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venuesNorm = venuesNorm.loc[:, updtCol]
venuesNorm['venue.categories'] = venuesNorm.apply(getData, axis=1)
venuesNorm.columns = [col.split(".")[-1] for col in venuesNorm.columns]
venuesNorm

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if sys.path[0] == '':


,name,categories,lat,lng
0,System Fitness,Gym,43.667171,-79.312733
1,Brett's Ice Cream,Ice Cream Shop,43.667222,-79.312831
2,Godspeed Brewery,Brewery,43.672620,-79.319228
3,Lake Inez,Asian Restaurant,43.672520,-79.320712
4,Woodbine Park,Park,43.664860,-79.315109
5,Leslieville Farmers Market,Farmers Market,43.664901,-79.319784
6,British Style Fish & Chips,Fish & Chips Shop,43.668723,-79.317139
7,Udupi Palace,Indian Restaurant,43.672480,-79.321275
8,Rorschach Brewing Co.,Brewery,43.663483,-79.319824
9,The Burger's Priest,Fast Food Restaurant,43.666731,-79.315556
